In [74]:
import numpy as np
import matplotlib.pyplot as plt
import ROOT as root
import simulacion as s
import os



In [75]:
#a integra
def riemann_superior(x_values,y_values,x_errors,y_errors): #riemann[0] is the value of the integral; riemann[1] is the error of the integral
    riemann = np.array([0.0,0.0])
    for i in range(len(x_values)-1):
        delta_x = x_values[i+1]-x_values[i]
        riemann[0] = riemann[0] + delta_x*y_values[i+1]
        #propagacion lineal de errores
        riemann[1] = riemann[1] + (y_values[i+1]**2)*(x_errors[i+1]**2+x_errors[i]**2) + (x_values[i+1]**2+x_values[i]**2)*(y_errors[i+1]**2)
    riemann[1]=np.sqrt(riemann[1])    
    return riemann

def trapecio(x,y):
    integral = 0
    for i in range(len(x)-1):
        delta_x = x[i+1]-x[i]
        integral = integral + delta_x*y[i] + 0.5*(y[i+1]-y[i])*delta_x
    return integral


def sorteo_montecarlo(x,y,xe,ye):
    #Inicializo los nuevos puntos x,y a sortear
    x_mc = np.zeros(len(x))
    y_mc = np.zeros(len(x))
    for i in range(len(x)):
        x_mc[i]=np.random.normal(x[i],xe[i])
        y_mc[i]=np.random.normal(y[i],ye[i])
    #Uso x_errors, y_errors inicializados a cero porque no influyen
    return x_mc,y_mc
def integral_montecarlo_sup(x_values,y_values,x_errors,y_errors):
    #calculo la integral con la suma de riemann
    integral_inicial = riemann_superior(x_values,y_values,x_errors,y_errors)[0]
    #Ahora se calcula la incertidumbre de la integral con metodo montecarlo
    integral = np.array([])
    #se usan 1000 interacciones con el siguiente procedimiento
    #1.sorteo de los nuevos valores de x e y para cada punto siguiendo una normal.
    #2.Los errores de parámetro en la función sorteo_montecarlo son irrelevantes.
    #3.Se calcula la integral del nuevo conjunto de puntos. Otra vez los errores introducidos de parámetro son irrelevantes
    #4.Se ordena la secuencia de integrales y se obtiene las posiciones entre las que se encuentra integral_inicial.
    # idem:  if  integral[i] < integral_inicial < integral[i+1] return posicion_inf = i, posicion_sup = i+1
    #5.Se calcula sigma+ y sigma-.
        #5.1     delta = 0.3413447460685429 * len(integral) gives the amount of data that constitutes 34.13%
        #5.2     sigma+ = integral[posicion_sup + int(delta)] - integral[posicion_sup]
    
    for i in range(1000):
        x,y=sorteo_montecarlo(x_values,y_values,x_errors,y_errors)
        integral = np.append(integral,riemann_superior(x,y,np.zeros(len(x)),np.zeros(len(x)))[0])
    integral = np.sort(integral)
    print(integral)
    pos_sup = 0
    pos_inf = 1000
    
    for i in range(len(integral)-1):
        if (integral[i]<integral_inicial<integral[i+1]) & integral_inicial<0.99:
            pos_inf = i
            pos_sup = i+1
    
    delta =  int(0.3413447460685429*len(integral))
    print(integral)
    sigma_plus = integral[pos_sup+delta] - integral[pos_sup]
    sigma_minus = integral[pos_inf] - integral[pos_inf-delta] 
    sigma = 0.5*(sigma_minus + sigma_plus)      
    return integral_inicial,sigma

def montecarlo_trapecio(x,y,xe,ye):
    integral_inicial = trapecio(x,y)
    integral = np.array([])
    for i in range(1000):
        x_mc,y_mc=sorteo_montecarlo(x,y,xe,ye)
        integral = np.append(integral,trapecio(x_mc,y_mc)) #Da igual que valores de xe,ye tomememos. De hecho creo que lo voy a borrar
    integral = np.sort(integral)

    '''for i in range(len(integral)-1):
        if integral[i]<integral_inicial<integral[i+1]:
            pos_inf = i
            pos_sup = i+1
    delta =  int(0.3413447460685429*len(integral))
    
    sigma_plus = integral[pos_sup+delta] - integral[pos_sup]
    sigma_minus = integral[pos_inf] - integral[pos_inf-delta] 
    sigma = 0.5*(sigma_minus + sigma_plus)      '''
    sigma = np.std(integral)
    return integral_inicial,sigma





In [76]:
'''
Objetivo: Ajustar x_values y_values x_errors y_errors a una funcion logistica de dos parametros
    y(x) == 0.5*(1 + 1/(1+exp([0]+x*[1]))) 
    [0], [1] son los parametros
Funciones necesarias:
    1. linealizar(x,y,xe,ye) transforma los datos originales para aplicar un ajuste lineal
        --SOLO linealiza los datos donde 0.
        
            
            '''

'\nObjetivo: Ajustar x_values y_values x_errors y_errors a una funcion logistica de dos parametros\n    y(x) == 0.5*(1 + 1/(1+exp([0]+x*[1]))) \n    [0], [1] son los parametros\nFunciones necesarias:\n    1. linealizar(x,y,xe,ye) transforma los datos originales para aplicar un ajuste lineal\n        --SOLO linealiza los datos donde 0.\n        \n            \n            '

**CURVA ROC --- INTEGRAL CURVA ROC --- AJUSTE CURVA AREA-CONTRASTE**

In [77]:
''' Voy a crear dos funciones principales:
1. curvaroc(n_pix,nct,t,k)
Input: modo de adquisicion
Output: Datos(x,y,xe,ye) y gráfica curva roc

2. areacontraste(n_pix,nct,t)
Input: modo de aquisicion salvo k (ahora es una variable)
Output: Objeto TGraphErrors y gráfica curva area contraste
---------de momento no realizo en ajuste dentro de esta funcion.

Tambien he incluido las siguientes funciones secundarias:
3. ajuste_ac(x,y,xe,ye)
Input: valores de la curva area-contraste
Output: parametros a,b del ajuste a la curva logistica
4. dibujarroc(x,y,xe,ye)
5. dibujarAC(x,y,xe,ye) ---- sirven para dibujar y guardar en un archivo las curvas roc y area-contraste
'''

def curvaroc(N_pix,nct,t,k,dibujar=False): #de momento el filtro es f1

    file_path = "./histogramas2/64x64/"+str(nct)+"mc/"
    name_fvn = s.histogram_name(N_pix,nct,1)
    name_ffn = s.histogram_name(N_pix,nct,1,t,k)
    file_fvn = root.TFile(file_path+name_fvn)
    file_ffn = root.TFile(file_path+name_ffn)
    hist_fvn = file_fvn.Get(name_fvn)
    hist_ffn = file_ffn.Get(name_ffn)
    fvn = hist_fvn.GetCumulative()
    ffn = hist_ffn.GetCumulative()
    nbins = hist_fvn.GetNbinsX() #nbins==400 Se debería aumentar?
    x,y = np.zeros(400),np.zeros(400)
    xe,ye = s.incertidumbre(ffn,10000),s.incertidumbre(fvn,10000)
    for i in range(400):
        x[i] = ffn.GetBinContent(i)
        y[i] = fvn.GetBinContent(i)
    if dibujar==True: dibujarroc(x,y,xe,ye,root.kBlue)
    #se podria plantear la opcion de devolver un .cv con los datos
    return x,y,xe,ye

def areacontraste(N_pix,nct,t,dibujar=False): #k no es input, es una variable de output
    k_list=[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10]
    integral_list=[]
    error_list=[]
    for k in k_list:
        x,y,xe,ye = curvaroc(N_pix,nct,t,k)
        integral,error = montecarlo_trapecio(x,y,xe,ye)
        integral_list.append(integral)
        error_list.append(error)
    x = np.array(k_list)
    xe = np.zeros(len(x))
    y = np.array(integral_list)
    ye = np.array(error_list)
    #dibujarAC(x,y,xe,ye,root.kRed)
    return x,y,xe,ye
    
def ajuste_ac(x,y,xe,ye):
    #1. Transformar los datos para realizar un ajuste lineal.
    x_fit,y_fit = np.zeros(len(x)),np.zeros(len(x))
    for i in range(len(x)):
        if 0.51<y[i]<0.99: #No consideros los puntos en los extremos saturados
            x_fit[i] = x[i] #x no cambia
            y_fit[i] = np.log(1/(2*y[i]-1)-1) #funcion de ajuste invertida -- falta considerar la transformacion de los errores
    x_fit = x_fit[x_fit!=0] #Elimino los elementos no nulos
    y_fit = y_fit[y_fit!=0]
    x_min = np.min(x_fit)
    x_max = np.max(x_fit)
    curvalineal = root.TGraph(len(x_fit),x_fit,y_fit)
    linearfit = root.TF1("linearfit","1++x",x_min,x_max) # a+b*x
    curvalineal.Fit(linearfit)
    linearfit = curvalineal.GetFunction("linearfit")
    a = linearfit.GetParameter(0)
    b = linearfit.GetParameter(1)
    asigma = linearfit.GetParError(0)
    bsigma = linearfit.GetParError(1)

    return a,b,asigma,bsigma



    
   
def dibujarroc(x,y,xe,ye,color): #los argumentos son los valores de fvn y ffn; color es un elemento kcolor
    roc = root.TGraphErrors(400)
    for i in range(400):
        roc.SetPoint(i,x[i],y[i])
        roc.SetPointError(i,xe[i],ye[i])
    roc.SetTitle("Receiver Operating Characteristic (ROC)")
    roc.GetXaxis().SetTitle("False Positive Rate")
    roc.GetYaxis().SetTitle("True Positive Rate")
    canvas = root.TCanvas('','',800,600)   
    roc.SetMarkerColor(color)
    roc.SetLineColor(color)
    roc.SetMarkerStyle(24)
    roc.Draw("APX") #los errores de momento son menores al tamaño de Marker
    canvas.SaveAs("roc_plot.png")

def dibujarAC(x,y,xe,ye,color): #los argumentos son los valores de fvn y ffn; color es un elemento kcolor
    roc = root.TGraphErrors(len(x))
    for i in range(len(x)):
        roc.SetPoint(i,x[i],y[i])
        roc.SetPointError(i,xe[i],ye[i])
    ajuste = root.TGraph(10000)
    x_fit = np.zeros(10000)
    y_fit = np.zeros(10000)
    a,b,asigma,bsigma = ajuste_ac(x,y,xe,ye)
    for i in range(10000):
        x_fit[i] = i*10/10000
        y_fit[i] = 0.5*(1+1/(1+np.exp(a+b*x_fit[i])))
        ajuste.SetPoint(i,x_fit[i],y_fit[i])
    ajuste.SetLineColor(color-5)
    roc.SetTitle("Curva AC: Area-Contraste")
    roc.GetXaxis().SetTitle("k - contraste")
    roc.GetYaxis().SetTitle("Aroc - Area curva roc")
    canvas = root.TCanvas('','',800,600)   
    roc.SetMarkerColor(color)
    roc.SetLineColor(color)
    roc.SetMarkerStyle(24)
    roc.Draw("AP") #los errores de momento son menores al tamaño del Marker
    ajuste.Draw("SAME")
    canvas.SaveAs("ac_plot.png")


 
x,y,xe,ye = areacontraste(64,20,3)  
a,b,asigma,bsigma = ajuste_ac(x,y,xe,ye)




'''x,y,xe,ye = areacontraste(64,20,4)
ajuste_ac(x,y,xe,ye)
dibujarAC(x,y,xe,ye,root.kBlue)'''

#dibujar los datos que sí son lineales tras la transformacion





'x,y,xe,ye = areacontraste(64,20,4)\najuste_ac(x,y,xe,ye)\ndibujarAC(x,y,xe,ye,root.kBlue)'

****************************************
Minimizer is Linear / Migrad
Chi2                      =   0.00125404
NDf                       =            1
p0                        =      5.35047   +/-   0.0778536   
p1                        =     -3.72163   +/-   0.0500807   


**Figura 2.3 - Comparacion**

In [78]:
def figura23(color):
    N_pix = 64
    nct = 20
    t_list = [1,2,3,4,5]
    curvas = []
    ajustes = []
    for t in t_list:
        x,y,xe,ye = areacontraste(N_pix,nct,t)
        ac = root.TGraphErrors(len(x))
        for i in range(len(x)):
            ac.SetPoint(i,x[i],y[i])
            ac.SetPointError(i,xe[i],ye[i])
        ajuste = root.TGraph(10000)
        x_fit = np.zeros(10000)
        y_fit = np.zeros(10000)
        a,b,asigma,bsigma = ajuste_ac(x,y,xe,ye)
        for i in range(10000):
            x_fit[i] = i*10/10000
            y_fit[i] = 0.5*(1+1/(1+np.exp(a+b*x_fit[i])))
            ajuste.SetPoint(i,x_fit[i],y_fit[i])
        curvas.append(ac)
        ajustes.append(ajuste)
    
    ajustes[0].SetLineColor(color)
    curvas[0].SetTitle("Curva AC: Area-Contraste")
    curvas[0].GetXaxis().SetTitle("k - contraste")
    curvas[0].GetYaxis().SetTitle("Aroc - Area curva roc")
    canvas = root.TCanvas('','',800,600)   
    curvas[0].SetMarkerColor(color)
    curvas[0].SetLineColor(color)
    curvas[0].SetMarkerStyle(24)
    #if t==1:ac.Draw("AP") #los errores de momento son menores al tamaño del Marker
    curvas[0].Draw("AP")
    ajustes[0].Draw("SAME")
    for i in range(len(t_list)-1):
        curvas[i+1].SetLineColor(color)
        curvas[i+1].SetMarkerColor(color)
        curvas[i+1].SetMarkerStyle(24)
        ajustes[i+1].SetLineColor(color)
        curvas[i+1].Draw("P")
        ajustes[i+1].Draw("SAME")
    canvas.Draw()
    canvas.SaveAs("figura23.png")
figura23(root.kBlack)    

****************************************
Minimizer is Linear / Migrad
Chi2                      =      0.13479
NDf                       =           12
p0                        =      6.18163   +/-   0.0922873   
p1                        =     -1.01036   +/-   0.0140533   
****************************************
Minimizer is Linear / Migrad
Chi2                      =    0.0782356
NDf                       =            4
p0                        =      5.70566   +/-   0.160911    
p1                        =     -2.51727   +/-   0.0668626   
****************************************
Minimizer is Linear / Migrad
Chi2                      =   0.00125404
NDf                       =            1
p0                        =      5.35047   +/-   0.0778536   
p1                        =     -3.72163   +/-   0.0500807   
****************************************
Minimizer is Linear / Migrad
Chi2                      =    0.0468549
NDf                       =            2
p0                  

Info in <TCanvas::Print>: png file figura23.png has been created


**CURVA CONTRASTE - DETALLE**

In [82]:
'''Voy a crear una funcion principal:
def contrastedetalle(Npix,nct):
    Input:: Modo de adquisicion (Npix,nct) Numero de pix y numero total de cuentas
    Output:: x,y,xe,ye (Datos de la funcion contraste - detalle)

Para cada tamaño t::
    1. Llamar a la funcion area-contraste para ca
    2. Ajuste area-contraste para cada tamaño t
    3. Invertir el ajuste, calcular el valor k_50%
    4. devolver (t,k_50%)
    5. calcular usando montecarlo el error de k_50%
    6. devolver x,y,xe,ye -- x=t^2  y=k_50%  xe=0  ye=errork_50%

        '''

def contrastedetalle(Npix,nct):
    t_list = np.array([1,2,3,4,5])
    k_list = np.zeros(len(t_list))
    sigma_list = np.zeros(len(t_list))
    for i in range(len(t_list)):
        x,y,xe,ye = areacontraste(Npix,nct,t_list[i],False)
        a,b,asigma,bsigma=ajuste_ac(x,y,xe,ye)
        aroc = 0.75 #Esto puede ser modificado a 0.8 -- 0.9
        if -0.01<b<0.01: break
        k = (1/b) * (-a + np.log(1/(2*aroc-1)-1))
        #implemento el montecarlo directamente
        montecarlo=np.zeros(1000)
        for j in range(1000):
            a_mc = np.random.normal(a,asigma)
            b_mc = np.random.normal(b,bsigma)
            if abs(b_mc)>0.01:
                k_mc = (1/b_mc)* (-a_mc + np.log(1/(2*aroc-1)-1))
            montecarlo[j]=k_mc
        k_list[i] = k
        sigma_list[i] = np.std(montecarlo)
    return t_list**2,k_list,sigma_list


def dibujarCD(x,y,xe,ye,color): #los argumentos son los valores de fvn y ffn; color es un elemento kcolor
    roc = root.TGraphErrors(len(x))
    for i in range(len(x)):
        roc.SetPoint(i,x[i],y[i])
        roc.SetPointError(i,xe[i],ye[i])
    roc.SetTitle("Curva CD: Contraste-Detalle")
    roc.GetXaxis().SetTitle("t^{2} - tamano del defecto (detalle)")
    roc.GetYaxis().SetTitle("k_50 - contraste")
    canvas = root.TCanvas('','',800,600)   
    roc.SetMarkerColor(color)
    roc.SetLineColor(color)
    roc.SetMarkerStyle(24)
    roc.Draw("AP") #los errores de momento son menores al tamaño del Marker
    canvas.SaveAs("cd_plot3.png")
x,y,ye=contrastedetalle(64,20)
xe = np.zeros(len(x))
dibujarCD(x,y,xe,ye,root.kRed)


****************************************
Minimizer is Linear / Migrad
Chi2                      =      0.13479
NDf                       =           12
p0                        =      6.18163   +/-   0.0922873   
p1                        =     -1.01036   +/-   0.0140533   
****************************************
Minimizer is Linear / Migrad
Chi2                      =    0.0782356
NDf                       =            4
p0                        =      5.70566   +/-   0.160911    
p1                        =     -2.51727   +/-   0.0668626   
****************************************
Minimizer is Linear / Migrad
Chi2                      =   0.00125404
NDf                       =            1
p0                        =      5.35047   +/-   0.0778536   
p1                        =     -3.72163   +/-   0.0500807   
****************************************
Minimizer is Linear / Migrad
Chi2                      =    0.0468549
NDf                       =            2
p0                  

Info in <TCanvas::Print>: png file cd_plot3.png has been created
